**Pattern 1** — Dedup by ID (Keep First Row)

**Problem:**
You have a list of records where the same business key (id) appears multiple times because of retries / replays. You want to keep only the first occurrence of each id.

**Task:**
Given a list of dict rows, remove duplicate ids while preserving the original order of the first appearance.

In [0]:
rows = [
    {"id": 101, "event": "click",  "ts": "2025-11-29T10:00:00"},
    {"id": 102, "event": "view",   "ts": "2025-11-29T10:00:05"},
    {"id": 101, "event": "click",  "ts": "2025-11-29T10:00:10"},  # duplicate id
    {"id": 103, "event": "signup", "ts": "2025-11-29T10:01:00"},
    {"id": 102, "event": "view",   "ts": "2025-11-29T10:01:05"},  # duplicate id
]


In [0]:
"""
Idea (Hash-Set on ID)
Maintain a set `seen` of IDs we've already output.

Scan rows in order:
- If row['id'] not in seen → add to output list and mark id as seen.
- If already in seen → skip (duplicate).

This keeps the FIRST occurrence and preserves input order.
"""

def dedup_by_id_keep_first(rows):
    seen = set()
    out = []
    for row in rows:
        rid = row["id"]
        if rid not in seen:
            seen.add(rid)
            out.append(row)
    return out


deduped = dedup_by_id_keep_first(rows)
# [
#   {"id": 101, "event": "click",  "ts": "2025-11-29T10:00:00"},
#   {"id": 102, "event": "view",   "ts": "2025-11-29T10:00:05"},
#   {"id": 103, "event": "signup", "ts": "2025-11-29T10:01:00"},
# ]

**Why this pattern (Dedup by ID)?**

Exact DE use case: fact-table replays, idempotent ingestion, upsert pre-processing.

Works even if payload changes for the same id — we only care about first occurrence.

Order-preserving, which matters for time-ordered logs or append-only streams.

**Time & Space Complexity**

Time: One pass over rows. Each set lookup/insert is amortized O(1) → O(n).

Space: seen can hold up to all distinct IDs, out holds up to all unique rows → O(n) extra space in worst case.

**Pattern 2** — Dedup by Entire Row (Exact Row Equality)

**Problem:**
You have fully duplicated rows (same all fields), possibly from double loading, file replays, or idempotent write issues. You want to keep one copy of each unique row.

**Task:**
Given a list of dict rows, remove exact duplicates where all key–value pairs match.

In [0]:
rows = [
    {"user": "alice", "event": "click", "ts": "2025-11-29T10:00:00"},
    {"user": "alice", "event": "click", "ts": "2025-11-29T10:00:00"},  # full duplicate
    {"user": "bob",   "event": "view",  "ts": "2025-11-29T10:00:05"},
    {"user": "alice", "event": "click", "ts": "2025-11-29T10:00:00"},  # full duplicate
]

In [0]:
"""
Idea (Canonical Tuple for Dict Row)
Dicts are not hashable, so we cannot put them directly in a set.

Trick:
- Convert each row dict into a *canonical* hashable form:
  tuple(sorted(row.items()))
- This normalizes key order: {"a":1,"b":2} and {"b":2,"a":1} map
  to the same tuple.
- Use a set of these tuples to track which full rows we've seen.

If canonical_row not in seen:
    add to seen and append original row to output.
"""

def dedup_by_entire_row(rows):
    seen = set()
    out = []
    for row in rows:
        key = tuple(sorted(row.items()))
        if key not in seen:
            seen.add(key)
            out.append(row)
    return out


deduped = dedup_by_entire_row(rows)
# [
#   {"user": "alice", "event": "click", "ts": "2025-11-29T10:00:00"},
#   {"user": "bob",   "event": "view",  "ts": "2025-11-29T10:00:05"},
# ]


**Why this pattern (Dedup by Entire Row)?**

Used when no single ID uniquely identifies a record, but the entire row should be treated as the key.

Good for raw ingested logs or pre-join staging tables where duplicates are byte-for-byte identical.

Canonicalization via sorted(row.items()) ensures different key orders still dedup to the same logical row.

**Time & Space Complexity**

Let:

n = number of rows

k = average number of fields per row

Time: For each row, we sort its items() (O(k log k)) and insert into a set (O(1) amortized).
→ Overall ~ O(n · k log k).
In practice, k is usually small and treated as constant → often seen as O(n) in DE discussions.

Space: seen stores up to one canonical tuple per distinct row and out stores unique rows
→ O(n) extra space in the worst case.